### Cuidados!
 
1. Whatsapp não gosta de nenhum tipo de automação
2. Isso pode dar merda, já to avisando
3. Isso não é o uso da API oficial do Whatsapp, o próprio whatsapp tem uma API oficial. Se o seu objetivo é fazer envio em massa ou criar aqueles robozinhos que respondem automaticamente no whatsapp, então use a API oficial
4. Meu objetivo é 100% educacional

### Dito isso, bora automatizar o envio de mensagens no Whatsapp

- Vamos usar o Selenium (vídeo da configuração na descrição)
- Temos 1 alternativa muito boa:
- Usar o https://web.whatsapp.com/send?phone=numero&text=texto (mais fácil, mais seguro, mas mais demorado)


In [1]:
import os
import time
import urllib
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

navegador = webdriver.Chrome(options=options)

def encontrar_elemento_botao(driver, metodo, seletor, tempo=20):
    return WebDriverWait(driver, tempo).until(EC.element_to_be_clickable((metodo, seletor)))


navegador.get("https://web.whatsapp.com")

## Importando a base de dados (site do whatsapp já carregado)

In [2]:
tabela_envio_mensagens = pd.read_excel("Envios.xlsx")

## Enviando mensagem para cada pessoa presente na base de dados

In [3]:
for linha in tabela_envio_mensagens.index:
    # armazenar informações
    nome = tabela_envio_mensagens.loc[linha, "nome"]
    mensagem = tabela_envio_mensagens.loc[linha, "mensagem"]
    arquivo_anexo = tabela_envio_mensagens.loc[linha, "arquivo"]
    telefone = tabela_envio_mensagens.loc[linha, "telefone"]

    texto = mensagem.replace("fulano", nome)
    texto = urllib.parse.quote(texto)

    # enviar mensagem para cada pessoa
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={texto}"

    try:
        navegador.get(link)
        
        # aguardar até que a página seja carregada (elemento com id='side' apareça) ou timeout
        # while len(navegador.find_elements(By.ID, "side")) < 1: # -> Lista for vazia (elemento não existe ainda)
        #     time.sleep(1)
        WebDriverWait(navegador, 30).until(EC.presence_of_element_located((By.ID, "side")))
        time.sleep(3)

        # verificando se o número é inválido
        alerta_erro_numero = navegador.find_elements(By.XPATH, '//div[contains(@data-testid, "alert-error")]')

        if len(alerta_erro_numero) == 0:
            # enviar a mensagem
            btn_enviar = encontrar_elemento_botao(navegador, By.XPATH, '//button[@data-testid="compose-btn-send"]')
            btn_enviar.click()
            
            # verificando se há anexo, caso sim, enviar
            if arquivo_anexo != "N":
                caminho_completo_arquivo = os.path.abspath(f"arquivos/{arquivo_anexo}")
                encontrar_elemento_botao(navegador, By.XPATH, '//div[@title="Anexar"]').click()
                input_arquivo = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//input[@accept="*"]')))
                input_arquivo.send_keys(caminho_completo_arquivo)

                # aguardar carregamento da prévia do arquivo e clicar em enviar
                time.sleep(10)
                encontrar_elemento_botao(navegador, By.XPATH, '//span[@data-icon="send"]').click()

            time.sleep(5)
        else:
            print(f"Número inválido: {telefone} -> {nome}")
    except Exception as e:
        print(f'Erro ao enviar mensagem para {nome} - {telefone}: {e}')
    # finally:
    #     time.sleep(4)
    #     navegador.quit()


Erro ao enviar mensagem para Micheli Oliveira - 48996501704: Message: 
Stacktrace:
0   chromedriver                        0x0000000100b036c8 cxxbridge1$str$ptr + 2791212
1   chromedriver                        0x0000000100afbc9c cxxbridge1$str$ptr + 2759936
2   chromedriver                        0x000000010064de30 cxxbridge1$string$len + 92928
3   chromedriver                        0x0000000100695170 cxxbridge1$string$len + 384576
4   chromedriver                        0x00000001006d65f8 cxxbridge1$string$len + 651976
5   chromedriver                        0x00000001006892fc cxxbridge1$string$len + 335820
6   chromedriver                        0x0000000100ac86c4 cxxbridge1$str$ptr + 2549544
7   chromedriver                        0x0000000100acb988 cxxbridge1$str$ptr + 2562540
8   chromedriver                        0x0000000100aa871c cxxbridge1$str$ptr + 2418560
9   chromedriver                        0x0000000100acc1e8 cxxbridge1$str$ptr + 2564684
10  chromedriver              